In [ ]:
# Define ADLS Gen2 Storage Account Name
# IMPORTANT: Replace 'earthquakedataluke' with your actual ADLS Gen2 storage account name for interactive testing.
storage_account_name = "earthquakedataluke" # Replace with your storage account name

# Uncomment the following lines for Azure Data Factory (ADF) execution:
# dbutils.widgets.text("storage_account_name", "", "Storage Account Name")
# storage_account_name = dbutils.widgets.get("storage_account_name")

print(f"Using storage_account_name: {storage_account_name}")

In [ ]:
from datetime import date, timedelta

# Remove this before running Data Factory Pipeline
start_date = date.today() - timedelta(1)

bronze_adls = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/"
silver_adls = f"abfss://silver@{storage_account_name}.dfs.core.windows.net/"


In [ ]:
''' Data Factory
dbutils.widgets.text("storage_account_name", "", "Storage Account Name")
storage_account_name_direct = dbutils.widgets.get("storage_account_name") # If passed directly by ADF
import json

# Retrieve the bronze_params directly as a widget
bronze_params = dbutils.widgets.get("bronze_params")
print(f"Raw bronze_params: {bronze_params}")

# Parse the JSON string
output_data = json.loads(bronze_params)
storage_account_name_param = output_data.get("storage_account_name", "") # From Bronze params

# Determine storage_account_name to use (prefer direct ADF param if available)
if 'storage_account_name_direct' in locals() and storage_account_name_direct:
    storage_account_name = storage_account_name_direct
elif 'storage_account_name_param' in locals() and storage_account_name_param:
    storage_account_name = storage_account_name_param
else:
    # Fallback or error if neither is available and it's required
    print("Warning: storage_account_name not found in ADF params or bronze_params. Using pre-defined or default.")
    # Ensure storage_account_name is defined, e.g. from an earlier cell for interactive, or raise error
    if 'storage_account_name' not in locals():
       storage_account_name = "YOUR_DEFAULT_STORAGE_ACCOUNT_NAME_HERE" # Should be set by widget cell in interactive
print(f"Storage account name for ADF context: {storage_account_name}")

# Access individual variables
start_date = output_data.get("start_date", "")
end_date = output_data.get("end_date", "")
# Update ADLS paths if they are also sourced from bronze_params, using the determined storage_account_name
bronze_adls = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/"
silver_adls = f"abfss://silver@{storage_account_name}.dfs.core.windows.net/"
gold_adls = f"abfss://gold@{storage_account_name}.dfs.core.windows.net/" # Assuming gold_adls might also be needed

print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")
'''

In [ ]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [ ]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

### Data Validation and Null Handling
- Null values for `longitude`, `latitude`, and `time` are preserved as `null`. Downstream processes should be prepared to handle these nulls appropriately.
- The original source (USGS API) may provide nulls or omit fields, which Spark will interpret as null when reading the JSON.

In [ ]:
df

In [ ]:
df.head()

In [ ]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [ ]:
df

In [ ]:
df.head()

In [ ]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [ ]:
df.head()

In [ ]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/" 

In [ ]:
# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)

In [ ]:
''' Data Factory
# Also pass storage_account_name to Gold notebook if needed, or ensure ADF passes it to Gold directly
output_to_gold = {
    "silver_output_path": silver_output_path,
    "storage_account_name": storage_account_name, # Pass it along
    "start_date": start_date # Pass start_date for partitioning or logging if Gold needs it
}
dbutils.notebook.exit(json.dumps(output_to_gold))
'''